In [1]:
import os 
import sys
import pickle

import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors

In [2]:
def find_closest_vector(v, M, n = 1):
    return np.flip(np.argsort(np.dot(M, v)))[:n] if n != 1 else np.argmax(np.dot(M,v))


In [3]:
path = "/data1/malto/csavelli/aligned_subwords_fasttext/"

ita_aligned = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.it.align.vec")
eng_aligned = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.en.align.vec")
#ita_wiki = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.it.bin") 
#eng_wiki = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.en.bin")

Look if the normal words are aligned with the same words of fastText. 

In [4]:
lang = "it"

with open(f"/data1/malto/csavelli/aligned_subwords_fasttext/res/{lang}/wiki.{lang}.pkl", "rb") as f:
    ita_new = pickle.load(f)

lang = "en"

with open(f"/data1/malto/csavelli/aligned_subwords_fasttext/res/{lang}/wiki.{lang}.pkl", "rb") as f:
    eng_new = pickle.load(f)

same language

In [5]:
idx = find_closest_vector(ita_new["casa"], ita_aligned.vectors)

ita_aligned.index_to_key[idx], idx

('casa', 228)

multi language

In [ ]:
# given a vector, find in a matrix the closest vector to it
# VALUTA DI USARE get_vector() per ottenere il vettore di una parola

idx = find_closest_vector(ita_new["ciao"], eng_new.vectors)
idx2 = find_closest_vector(ita_new["ciao"], eng_aligned.vectors)
eng_new.index_to_key[idx], idx, eng_aligned.index_to_key[idx2], idx2

In [ ]:
idx = find_closest_vector(ita_new["casa"], eng_new.vectors)
eng_new.index_to_key[idx], idx

In [ ]:
idx = find_closest_vector(ita_new["gatto"], eng_new.vectors)
eng_new.index_to_key[idx], idx

In [ ]:
idx = find_closest_vector(ita_new["papero"], eng_new.vectors)
eng_new.index_to_key[idx], idx

In [ ]:
idx = find_closest_vector(ita_aligned["ciao"], eng_aligned.vectors)
print(eng_aligned.index_to_key[idx], idx)

idx = find_closest_vector(ita_aligned["casa"], eng_aligned.vectors)
print(eng_aligned.index_to_key[idx], idx) 

idx = find_closest_vector(ita_aligned["gatto"], eng_aligned.vectors)   
print(eng_aligned.index_to_key[idx], idx)

idx = find_closest_vector(ita_aligned["papero"], eng_aligned.vectors)
print(eng_aligned.index_to_key[idx], idx)

In [ ]:
word = "balenottero"

idx = find_closest_vector(ita_new[word], eng_new.vectors)
ita_new.has_index_for(word), eng_new.index_to_key[idx], idx

In [ ]:
word = "pomodoriniq"

idx = find_closest_vector(ita_new[word], eng_new.vectors)
ita_new.has_index_for(word), eng_new.index_to_key[idx], idx

In [ ]:
word = "alberelo"

idx = find_closest_vector(ita_new[word], ita_new.vectors)
key = ita_new.index_to_key[idx]
print(key, idx)
print()
idx = find_closest_vector(ita_new[key], eng_new.vectors)
print(eng_new.index_to_key[idx], idx)

In [ ]:
word = "orchetti"

idx = find_closest_vector(ita_new[word], eng_new.vectors, 10)
for i in idx: 
    print(ita_new.has_index_for(word), eng_new.index_to_key[i], i)

In [ ]:
idx = find_closest_vector(ita_aligned[word], eng_aligned.vectors)
print(eng_aligned.index_to_key[idx], idx)

In [ ]:
word = "topolino"

idx = find_closest_vector(ita_new[word], eng_new.vectors)
ita_new.has_index_for(word), eng_new.index_to_key[idx], idx

5: take random words from ita_new and watch the closest in eng_new and see if the pattern is the same with fastText

In [ ]:
# generate 2000 random indexes between 0 and ita_new.index_to_key length 

np.random.seed(42)

idx = np.random.randint(0, 4000, 2000)

for i in idx:
    ita_word = ita_new.index_to_key[i]
    eng_word_ew = eng_new.index_to_key[find_closest_vector(ita_new[ita_word], eng_new.vectors)]
    eng_word_aw = eng_aligned.index_to_key[find_closest_vector(ita_aligned[ita_word], eng_aligned.vectors)]

    if ita_new.index_to_key[i] != ita_aligned.index_to_key[i]:
        print("PROBLEM: ", i)
    
    if eng_word_ew != eng_word_aw:
        print(ita_word, eng_word_ew, eng_word_aw)

# not working

In [ ]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors
import pickle

def find_matrix(lang, step=1000):
    
    print("LANG", lang)
    dict = {}

    # not aligned
    try:
        src = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.bin") 
        print("Loaded fastText vectors")
    except:
        print("Going to 'vec'")
        src = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.vec")
    dst = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.{lang}.align.vec") # aligned
    
    if src.index_to_key != dst.index_to_key:
        print("src and dst vocabularies differ. ")
        print("src", len(src))
        print("dst", len(dst))
        print("in src, not in dst", set(src.index_to_key) - set(dst.index_to_key))
        print("in dst, not in src", set(dst.index_to_key) - set(src.index_to_key))
    
    dict["missing_elements"] = [set(src.index_to_key) - set(dst.index_to_key), set(dst.index_to_key) - set(src.index_to_key)] # missing words in common vocabulary

    vocab = sorted(list(set(src.index_to_key) & set(dst.index_to_key)))
        
    Y = dst[vocab]
    X = src[vocab]

    W_ = np.linalg.pinv(X) @ Y

    prod = (X @ W_)
    prod = prod / np.linalg.norm(prod, axis=1).reshape(-1,1)

    dict["MSE"] = np.square(np.subtract(prod, Y)).sum(axis=1).mean() # mean squared error

    error_couples = []
    right_values = np.array([])

    for i in range(0, len(prod), step):
            M = (prod[i:i+step] @ Y.T)
            v = M.argmax(axis=1)

            # sum of the diagonal
            right_values = np.concatenate((right_values, np.diagonal(M[:,v])))
            for j in range(len(v)):
                if v[j] != i+j: # check that the most vector is the word itself
                    print("words do not match", i+j, v[j], M[j,v[j]])
                    print("instead the right word should be ", M[j,j])
                    error_couples.append((i+j, v[j], M[j,v[j]], M[j,j]))

            print(i, "/", len(prod), "done") if i % 50_000 == 0 else None
    
    dict["accuracy"] = (right_values.mean(), right_values.std()) # values of the diagonal of the matrix
    dict["n_errors"] = len(error_couples) # number of errors

    src.vectors = src.vectors @ W_
    src.vectors_ngrams = src.vectors_ngrams @ W_
    src.vectors = src.vectors / np.linalg.norm(src.vectors, axis=1).reshape(-1,1)
    src.vectors_ngrams = src.vectors_ngrams / np.linalg.norm(src.vectors_ngrams, axis=1).reshape(-1,1)

    return src, X, Y, W_, right_values, error_couples, dict

            
lang = "af"         
src, X, Y, W_, right_values, error_couples, dict = find_matrix(lang)

In [ ]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors
import pickle

import os 

def create_bin(lang):
    
    print("LANG", lang)

    # not aligned
    try:
        src = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.bin") 
        print("Loaded fastText vectors")
    except:
        print("Going to 'vec'")
        src = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.vec")
    
    path = "W/"
    file = f"{lang}.pkl"
    with open(os.path.join(path, file), "rb") as f:
        W_ = pickle.load(f)

    src.vectors = src.vectors @ W_
    src.vectors_ngrams = src.vectors_ngrams @ W_
    src.vectors = src.vectors / np.linalg.norm(src.vectors, axis=1).reshape(-1,1)
    src.vectors_ngrams = src.vectors_ngrams / np.linalg.norm(src.vectors_ngrams, axis=1).reshape(-1,1)

    return W_, src 

W1, src1 = create_bin("af")

In [ ]:
FastText

In [ ]:
src2 = KeyedVectors.load(f"/data1/malto/csavelli/aligned_subwords_fasttext/res/{lang}/wiki.{lang}.bin")
vectors_ngrams = np.load(f"/data1/malto/csavelli/aligned_subwords_fasttext/res/{lang}/wiki.{lang}.bin.vectors_ngrams.npy", allow_pickle=True)
vectors_vocab = np.load(f"/data1/malto/csavelli/aligned_subwords_fasttext/res/{lang}/wiki.{lang}.bin.vectors_vocab.npy", allow_pickle=True)

In [ ]:
(src.vectors == src1.vectors).all(), (src.vectors_ngrams == vectors_ngrams).all()

In [ ]:
src.save("test")

In [ ]:
bonk = KeyedVectors.load("test")
bonk.vectors == src.vectors, 

In [ ]:
plain.vectors = src.vectors
plain.vectors_ngrams = src.vectors_ngrams

In [ ]:
src, X, Y, W_, right_values, error_couples, dict = find_matrix(lang)

In [ ]:
# save src with pickle 

import pickle

path = "W/"
file = f"{lang}.pkl"

with open(f"test2", "wb") as f:
                pickle.dump((src), f)

In [ ]:
# load src with pickle
with open(f"test2", "rb") as f:
    bonk = pickle.load(f)

(bonk.vectors == src.vectors).all(), (bonk.vectors_ngrams == src.vectors_ngrams).all()  

In [ ]:
lang = "af"
with open(f"/data1/malto/csavelli/aligned_subwords_fasttext/res/{lang}/wiki.{lang}.pkl", "rb") as f:
    bonk2 = pickle.load(f)

(bonk2.vectors == src.vectors).all(), (bonk2.vectors_ngrams == src.vectors_ngrams).all()  

In [ ]:
src.save("test")

In [ ]:
test_model = KeyedVectors.load("test")
vectors_ngrams = np.load("test.vectors_ngrams.npy", allow_pickle=True)
vectors_vocab = np.load(f"test.vectors_vocab.npy", allow_pickle=True)

In [ ]:
vectors_ngrams.shape, vectors_vocab.shape, src.vectors_ngrams.shape, src.vectors.shape

In [ ]:
v = vectors_vocab[0]

for row in src.vectors:
    if (row == v).all():
        print("found")
        break

In [ ]:
src.vectors

In [ ]:
vectors_vocab

In [ ]:
src.vectors_ngrams == vectors_ngrams, src.vectors == vectors_vocab

In [ ]:
test_model.vectors_ngrams == vectors_ngrams, test_model.vectors == src.vectors